In [51]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn import preprocessing, model_selection, metrics
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor

from IPython.display import display # Allows the use of display() for DataFrames

import warnings
warnings.filterwarnings('ignore')

In [52]:
train_df = pd.read_csv('../input/santander-value-prediction-challenge/train.csv')

# Given the test.csv file is huge, reading which each time during development
# takes couple of minutes - making my development process slower.
# I am reading only first 100 rows during development.
test_df = pd.read_csv('../input/santander-value-prediction-challenge/test.csv', nrows=100)

# But in Kaggle Kernel, and before final submission
# comment-out the above line and un-comment below line to read the full train.csv
# test_df = pd.read_csv('../input/santander-value-prediction-challenge/test.csv')

train_df.head()

,ID,target,48df886f9,0deb4b6a8,34b15f335,a8cb14b00,2f0771a37,30347e683,d08d1fbe3,6ee66e115,...,3ecc09859,9281abeea,8675bec0b,3a13ed79a,f677d4d13,71b203550,137efaa80,fb36b89d9,7e293fbaf,9fc776466
0,000d6aaf2,38000000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
1,000fbd867,600000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
2,0027d6b71,10000000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
3,0028cbf45,2000000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
4,002a68644,14400000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0


In [ ]:
test_df.head()

Initial Observations looking at the above data

- Column name does not mean anything now, as they are all anonymized
- The dataframe is full of zero values.
- The dataset is a sparse tabular one refer [this](https://www.kaggle.com/c/santander-value-prediction-challenge/discussion/59128)

Target Variable:

First doing some scatter plot of the target variable to check for visible outliers.

In [53]:
print('Train rows and columns: ', train_df.shape)

# Keeping below line commented out as its huge 49,342 row file with 1gb size and so take longer to run each time
print('Test rows and columns: ', test_df.shape)

Train rows and columns:  (4459, 4993)


In [54]:
# Keeping below lines commented out during development

# plt.figure(figsize=(8,6))
# plt.scatter(range(train_df.shape[0]), np.sort(train_df['target'].values))
# plt.xlabel('index', fontsize=12)
# plt.ylabel('Target', fontsize=12)
# plt.title('Distribution of Target', fontsize=14)
# plt.show()

TO-DO - So there's not too much of outliers (visibly) but the distribution range is high. Now want to do a histogram

## Checking for missing / null values in data

In [55]:
print("All Features in Train data with NaN Values =", str(train_df.columns[train_df.isnull().sum() != 0].size) )
# print("All Features in Test data with NaN Values =", str(test_df.columns[train_df.isnull().sum() != 0].size) )

All Features in Train data with NaN Values = 0


## Remove constant columns from data

In [56]:
const_columns_to_remove = []
for col in train_df.columns:
    if col != 'ID' and col != 'target':
        if train_df[col].std() == 0:
            const_columns_to_remove.append(col)

# Now remove that array of const columns from the data
train_df.drop(const_columns_to_remove, axis=1, inplace=True)
test_df.drop(const_columns_to_remove, axis=1, inplace=True)

# Print to see the reduction of columns
print('Train rows and columns after removing constant columns: ', train_df.shape)

print('Following `{}` Constant Column\n are removed'.format(len(const_columns_to_remove)))
print(const_columns_to_remove)

Train rows and columns after removing constant columns:  (4459, 4737)
Following `256` Constant Column
 are removed
['d5308d8bc', 'c330f1a67', 'eeac16933', '7df8788e8', '5b91580ee', '6f29fbbc7', '46dafc868', 'ae41a98b6', 'f416800e9', '6d07828ca', '7ac332a1d', '70ee7950a', '833b35a7c', '2f9969eab', '8b1372217', '68322788b', '2288ac1a6', 'dc7f76962', '467044c26', '39ebfbfd9', '9a5ff8c23', 'f6fac27c8', '664e2800e', 'ae28689a2', 'd87dcac58', '4065efbb6', 'f944d9d43', 'c2c4491d5', 'a4346e2e2', '1af366d4f', 'cfff5b7c8', 'da215e99e', '5acd26139', '9be9c6cef', '1210d0271', '21b0a54cb', 'da35e792b', '754c502dd', '0b346adbd', '0f196b049', 'b603ed95d', '2a50e001c', '1e81432e7', '10350ea43', '3c7c7e24c', '7585fce2a', '64d036163', 'f25d9935c', 'd98484125', '95c85e227', '9a5273600', '746cdb817', '6377a6293', '7d944fb0c', '87eb21c50', '5ea313a8c', '0987a65a1', '2fb7c2443', 'f5dde409b', '1ae50d4c3', '2b21cd7d8', '0db8a9272', '804d8b55b', '76f135fa6', '7d7182143', 'f88e61ae6', '378ed28e0', 'ca4ba131e', 

## Remove Duplicate Columns

**I will be using the duplicated() function of pandas - here's how it works:**

Suppose the columns of the data frame are `['alpha','beta','alpha']`

`df.columns.duplicated()` returns a boolean array: a `True` or `False` for each column. If it is `False` then the column name is unique up to that point, if it is `True` then the column name is duplicated earlier. For example, using the given example, the returned value would be `[False,False,True]`. 

`Pandas` allows one to index using boolean values whereby it selects only the `True` values. Since we want to keep the unduplicated columns, we need the above boolean array to be flipped (ie `[True, True, False] = ~[False,False,True]`)

Finally, `df.loc[:,[True,True,False]]` selects only the non-duplicated columns using the aforementioned indexing capability. 

**Note**: the above only checks columns names, *not* column values.

In [57]:
train_df = train_df.loc[:,~train_df.columns.duplicated()]
print('Train rows and columns after removing duplicate columns: ', train_df.shape)

Train rows and columns after removing duplicate columns:  (4459, 4737)


## Handling Sparse data

**What is Sparse data**

As an example, let's say that we are collecting data from a device which has 12 sensors. And you have collected data for 10 days.

The data you have collected is as follows:
[![enter image description here][1]][1]

The above is an example of sparse data because most of the sensor outputs are zero. Which means those sensors are functioning properly but the actual reading is zero. Although this matrix has high dimensional data (12 axises) it can be said that it contains less information.

So basically, sparse data means that there are many gaps present in the data being recorded. For example, in the case of the sensor mentioned above, the sensor may send a signal only when the state changes, like when there is a movement of the door in a room. This data will be obtained intermittently because the door is not always moving. Hence, this is sparse data.

  [1]: https://i.stack.imgur.com/Af5IH.png


First lets have a look at or train_df data again, that how much of sparse data is there. And as we can see there are plenty of '0'

In [58]:
train_df.head()

,ID,target,48df886f9,0deb4b6a8,34b15f335,a8cb14b00,2f0771a37,30347e683,d08d1fbe3,6ee66e115,...,3ecc09859,9281abeea,8675bec0b,3a13ed79a,f677d4d13,71b203550,137efaa80,fb36b89d9,7e293fbaf,9fc776466
0,000d6aaf2,38000000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
1,000fbd867,600000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
2,0027d6b71,10000000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
3,0028cbf45,2000000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
4,002a68644,14400000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0


## Check and handle total memory of data

`get_dummies` pandas function converts categorical variables into indicator variables.

In [59]:
def print_memory_usage_of_df(df):
    bytes_per_mb = 0.000001
    memory_usage = round(df.memory_usage().sum() * bytes_per_mb, 3)
    print('Memory usage is ', str(memory_usage) + " MB")

print_memory_usage_of_df(train_df)
print(train_df.shape)

Memory usage is  168.978 MB
(4459, 4737)


In [60]:
dummy_encoded_train_df = pd.get_dummies(train_df)
dummy_encoded_train_df.shape

(4459, 9195)

In [61]:
print_memory_usage_of_df(dummy_encoded_train_df)

Memory usage is  188.825 MB


We see that the memory usage of the dummy_encoded_train_df data  frame is larger compared to the original, because now the number of columns have increased in the data frame.

##### So lets apply `sparse=True` if it reduces the memory-usages to some extent.

This parameter `sparse` defaults to False. If True the encoded columns are returned as **SparseArray**. By setting `sparse=True` we create a sparse data frame directly

In [62]:
dummy_encoded_sparse_train_df = pd.get_dummies(train_df, sparse=True)
dummy_encoded_sparse_train_df.shape

(4459, 9195)

In [63]:
print_memory_usage_of_df(dummy_encoded_sparse_train_df)


Memory usage is  168.965 MB


But looks like in this case the reduction in memory_size was not a huge amount. So lets try some other alternative

## [Pandas Sparse Structures](https://pandas.pydata.org/pandas-docs/stable/user_guide/sparse.html#sparse-data-structures)

Pandas provides data structures for efficient storage of sparse data. In these structures, zero values (or any other specified value) are not actually stored in the array. Rather, you can view these objects as being “compressed” where any data matching a specific value (NaN / missing value, though any value can be chosen, including 0) is omitted. The compressed values are not actually stored in the array.

Storing only the non-zero values and their positions is a common technique in storing sparse data sets.

This hugely reduces the memory usage of our data set and “compress” the data frame.

In our example, we will convert the one-hot encoded columns into SparseArrays, which are 1-d arrays where only non-zero values are stored.

In [64]:
def convert_df_to_sparse_array(df, exclude_columns=[]):
    df = df.copy()
    exclude_columns = set(exclude_columns)

    for (column_name, column_data) in df.iteritems():
        if column_name in exclude_columns:
            continue
        df[column_name] = pd.SparseArray(column_data.values, dtype='uint8')

    return df

# Now convert our earlier dummy_encoded_train_df with above function and check memory_size

# train_data_post_conversion_to_sparse_array = convert_df_to_sparse_array(dummy_encoded_train_df)
# print('Sparse Array Train_DF rows and columns: ', train_data_post_conversion_to_sparse_array.shape)
# print_memory_usage_of_df(train_data_post_conversion_to_sparse_array)

# Commenting the above out - for running the Notebook faster during my development

**We see the that the memory_usage is substantially reduced now**

### A warning on using df.iteritems()

The df.iteritems() iterates over columns and not rows. Generally iteration over dataframes is an anti-pattern, and something we should avoid, unless you want to get used to a lot of waiting.

### An eazier way to handle sparse data is just to drop it from the dataframe
like below code, I will do this for the sake of running this notebook faster
And then later check back which approach gives me better predictions

In [65]:
def drop_parse_from_df(df):
    column_list_to_drop_data_from = [i for i in df.columns if not i in ['ID', 'target'] ]
    for column in column_list_to_drop_data_from:
        if len(np.unique(df[column])) < 2:
            df.drop(column, axis=1, inplace=True)
            df.drop(column, axis=1, inplace=True)
    return df

# The same above function, if I wanted to do for 2 dataframes together.
# def drop_parse_from_df(df_1, df_2):
#     column_list_to_drop_data_from = [i for i in df_1.columns if not i in ['ID', 'target'] ]
#     for column in column_list_to_drop_data_from:
#         if len(np.unique(df_1[column])) < 2:
#             df_1.drop(column, axis=1, inplace=True)
#             df_2.drop(column, axis=1, inplace=True)
#     return df_1, df_2

train_df = drop_parse_from_df(train_df)
print('Rows and Columns in train_df after removing sparse ', format(train_df.shape))


Rows and Columns in train_df after removing sparse  (4459, 4737)


### Split data into Train and Test for Model Training

In [66]:
X_train = train_df.drop(['ID', 'target'], axis=1)

y_train = np.log1p(train_df['target'].values)

X_test = test_df.drop('ID', axis=1)
X_train_split, X_validation, y_train_split, y_validation = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [67]:
## LightGBM

In [68]:
def run_light_gbm(train_X, train_y, validation_X, validation_y, test_X):
    params = {
        "objective" : "regression",
        "metric" : "rmse",
        "num_leaves" : 40,
        "learning_rate" : 0.004,
        "bagging_fraction" : 0.6,
        "feature_fraction" : 0.6,
        "bagging_frequency" : 6,
        "bagging_seed" : 42,
        "verbosity" : -1,
        "seed": 42
    }


